CG construction

In [1]:
import csv, functools, json, warnings, re
import os, random
import numpy as np
import torch
from torch.utils.data import Dataset
from pymatgen.core import Element, Structure, Lattice
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

class GaussianDistance(object):
    def __init__(self, dmin, dmax, step, var=None):
        assert dmin < dmax
        assert dmax - dmin > step
        self.filter = np.arange(dmin, dmax + step, step)
        self.var = step if var is None else var

    def expand(self, distances):
        return np.exp(-(distances[..., np.newaxis] - self.filter) ** 2 / self.var ** 2)


class CIFData(Dataset):
    def __init__(self, id_prop_path, magmom_path, eigen_path, ofm_val_comp_path,
             max_num_nbr=12, radius=8, dmin=0, step=0.1, random_seed=123):


        # Load CSV
        with open(id_prop_path) as f:
            reader = csv.reader(f)
            self.id_prop_data = [row for row in reader]

        # Load magnetic moments
        with open(magmom_path) as f:
            self.magmom = json.load(f)

        # Load magnetic moments
        with open(eigen_path) as f:
            self.eigen1 = json.load(f)

        # Load magnetic moments
        with open(ofm_val_comp_path) as f:
            self.ofm_val_comp = json.load(f)

        random.seed(random_seed)
        random.shuffle(self.id_prop_data)

        self.gdf = GaussianDistance(dmin=dmin, dmax=radius, step=step)
        self.max_num_nbr = max_num_nbr
        self.radius = radius

    def __len__(self):
        return len(self.id_prop_data)

    def structure_from_str(self, cif_string):
        cif_lines = cif_string.strip().split('\n')

    # Parse lattice
        a, b, c = map(float, cif_lines[2].split()[2:5])
        alpha, beta, gamma = map(float, cif_lines[3].split()[1:4])

        species, coords = [], []

        reading_atoms = False
        for line in cif_lines:
            line = line.strip()
            if re.match(r'^\d+\s+\w+\s+[\d\.Ee+-]+\s+[\d\.Ee+-]+\s+[\d\.Ee+-]+', line):
                reading_atoms = True
            if not reading_atoms:
                continue
            try:
                parts = line.split()
                if len(parts) < 5:
                    continue
                sp = parts[1]
                a_frac, b_frac, c_frac = map(float, parts[2:5])
                species.append(sp)
                coords.append([a_frac, b_frac, c_frac])
            except Exception as e:
                warnings.warn(f"Skipping line due to error: {line} ({e})")
                continue

        lattice = Lattice.from_parameters(a, b, c, alpha, beta, gamma)
        return Structure(lattice, species, coords)


    def featurizer(self, element):
        min_max = [[1.00794, 244.0], [1.0, 118.0], [0.25, 2.6], [-0.72, 3.61272528]]
        lis = ['Atomic mass', 'Atomic no', 'Atomic radius', 'Electron affinity']
        ele = Element(element)
        try:
            fea = [float(str(ele.data[i]).split()[0]) for i in lis]
        except:
            fea = [ele.data['Atomic mass'], ele.data['Atomic no'],
                   ele.data['Atomic radius calculated'], ele.data['Electron affinity']]
        for i in range(4):
            fea[i] = (fea[i] - min_max[i][0]) / (min_max[i][1] - min_max[i][0])
        return fea

    @functools.lru_cache(maxsize=None)
    def __getitem__(self, idx):
        cif_id, cif_string, target = self.id_prop_data[idx]
        crystal = self.structure_from_str(cif_string)

        atom_fea = np.vstack([
            [self.magmom[site.species_string]] + self.ofm_val_comp[site.species_string]
            + self.featurizer(site.species_string)
            for site in crystal
        ])
        atom_fea = torch.Tensor(atom_fea)

        all_nbrs = crystal.get_all_neighbors(self.radius, include_index=True)
        all_nbrs = [sorted(nbrs, key=lambda x: x[1]) for nbrs in all_nbrs]
        nbr_fea_idx, nbr_fea = [], []

        for i, nbr in enumerate(all_nbrs):
            if len(nbr) < self.max_num_nbr:
                warnings.warn(f'{cif_id} atom {i} has only {len(nbr)} neighbors. Consider increasing radius.')
                nbr_fea_idx.append([x[2] for x in nbr] + [0] * (self.max_num_nbr - len(nbr)))
                nbr_fea.append([x[1] for x in nbr] + [self.radius + 1.] * (self.max_num_nbr - len(nbr)))
            else:
                nbr_fea_idx.append([x[2] for x in nbr[:self.max_num_nbr]])
                nbr_fea.append([x[1] for x in nbr[:self.max_num_nbr]])

        nbr_fea_idx = torch.LongTensor(nbr_fea_idx)
        nbr_fea = torch.Tensor(self.gdf.expand(np.array(nbr_fea)))
        target = torch.Tensor([float(target)])
        return (atom_fea, nbr_fea, nbr_fea_idx), target, cif_id



In [2]:
# 1. Instantiate dataset with exact file paths — edit these paths if your files are elsewhere.
dataset = CIFData(
    id_prop_path='my_new_data1/all_fim_3000.csv',                 # <-- your CSV of (id, cif_string, target)
    magmom_path='my_new_data1/magmom.json',             # <-- edit if located elsewhere
    eigen_path='my_new_data1/eigenvalues_bulk.json',    # <-- edit if located elsewhere
    ofm_val_comp_path='my_new_data1/ofm.json',          # <-- edit if located elsewhere
    max_num_nbr=12,
    radius=8,
    dmin=0,
    step=0.1,
    random_seed=123
)

# Quick sanity prints
print("Loaded dataset with", len(dataset), "entries.")
print("First CSV row preview:", dataset.id_prop_data[0][:2])  # id and first part of cif string


Loaded dataset with 3000 entries.
First CSV row preview: ['mp-1219989', 'Full Formula (Pr1 Fe1 Co4)\nReduced Formula: PrFeCo4\nabc   :   5.070117   5.070117   3.941409\nangles:  90.000000  90.000000 120.000008\npbc   :       True       True       True\nSites (6)\n  #  SP           a         b    c    magmom\n---  ----  --------  --------  ---  --------\n  0  Pr    0.666667  0.333333  0      -0.236\n  1  Fe    0         0         0       2.604\n  2  Co    0.333333  0.666667  0       1.615\n  3  Co    0.167998  0.335996  0.5     1.564\n  4  Co    0.664004  0.832002  0.5     1.342\n  5  Co    0.167998  0.832002  0.5     1.598']


CG-Vec Dimension expansion for tracing feature.

In [3]:
# ---------------------------
# Cell B (minimal): CG -> VEC vectorization using only original features
# - Species blocks: count + mean(node_features) + std(node_features)
# - Edge RBF summaries: mean & std per RBF bin across all edges
# - Global mean & std per node_feature
# No top-K slots, no compression beyond RBF bins.
# ---------------------------

import os, json, time, numpy as np, torch

# PARAMETERS (do not change unless you understand consequences)
id_prop_list = dataset.id_prop_data
RBF_step = 0.1   #  this matches your CIFData GaussianDistance.step
# We will use the RBF centers as constructed by GaussianDistance in dataset.gdf.filter
rbf_centers = dataset.gdf.filter
num_rbf = len(rbf_centers)  # number of gaussian bins (e.g., (radius - dmin)/step + 1 )

os.makedirs('cgvec_mapping', exist_ok=True)

# 1) global species set
global_species_set = set()
for row in id_prop_list:
    cif_id, cif_string, _ = row
    struct = dataset.structure_from_str(cif_string)
    for site in struct:
        global_species_set.add(site.species_string)
global_species = sorted(list(global_species_set))
n_species = len(global_species)

# 2) infer node dimension and rbf bins
(first_atom_fea, first_nbr_fea, first_nbr_idx), _, _ = dataset[0]
D_node = int(first_atom_fea.shape[1])   # should be 37
D_edge = int(first_nbr_fea.shape[2]) if (first_nbr_fea is not None and first_nbr_fea.numel()>0) else 0
print("D_node=", D_node, "num_rbf=", num_rbf, "inferred D_edge=", D_edge)

# 3) build mapping_global deterministically
mapping_global = []
cursor = 0
# species blocks
for s in global_species:
    block_len = 1 + D_node + D_node
    mapping_global.append({'type':'species_block','species':s,'start':cursor,'end':cursor+block_len})
    cursor += block_len
# edge RBF bins: each RBF bin -> mean & std (2 scalars)
for k in range(num_rbf):
    mapping_global.append({'type':'edge_rbf_bin','rbf_idx':int(k),'start':cursor,'end':cursor+2})
    cursor += 2
# global mean & std
mapping_global.append({'type':'global_atom_mean','start':cursor,'end':cursor + D_node}); cursor += D_node
mapping_global.append({'type':'global_atom_std','start':cursor,'end':cursor + D_node}); cursor += D_node

D_total = cursor
print("D_total=", D_total, "mapping entries=", len(mapping_global))

with open('cgvec_mapping/mapping_global.json','w') as f:
    json.dump(mapping_global, f, indent=2)
print("Saved mapping_global.json")

# 4) vectorize dataset (species blocks + rbf bin summaries + global stats)
N = len(dataset)
X = np.zeros((N, D_total), dtype=np.float32)
y = np.zeros(N, dtype=float)

t0 = time.time()
for i in range(N):
    (atom_fea, nbr_fea, nbr_idx), target, cif_id = dataset[i]
    struct = dataset.structure_from_str(dataset.id_prop_data[i][1])
    species_list = [site.species_string for site in struct]
    atom = atom_fea.detach().cpu().numpy()  # shape (n_atoms, D_node)

    parts = []
    # species blocks
    for s in global_species:
        idxs = [j for j,sp in enumerate(species_list) if sp==s]
        if len(idxs)==0:
            cnt = 0.0
            mean = np.zeros(D_node, dtype=float)
            std  = np.zeros(D_node, dtype=float)
        else:
            feats = atom[idxs]
            cnt = float(feats.shape[0])
            mean = feats.mean(axis=0)
            std  = feats.std(axis=0, ddof=0)
        parts.append(np.concatenate([[cnt], mean, std]).astype(float))

    # edge RBF bin summaries (mean & std per RBF bin across all edges)
    if nbr_fea is None or (isinstance(nbr_fea, torch.Tensor) and nbr_fea.numel()==0):
        # if no neighbor data, pad zeros for each bin
        for _ in range(num_rbf):
            parts.append(np.array([0.0, 0.0], dtype=float))
    else:
        flat = nbr_fea.detach().cpu().numpy().reshape(-1, nbr_fea.shape[2])  # shape (n_edges, num_rbf)
        if flat.size == 0:
            for _ in range(num_rbf):
                parts.append(np.array([0.0, 0.0], dtype=float))
        else:
            # per RBF bin compute mean & std across edges
            for k in range(num_rbf):
                col = flat[:, k]
                parts.append(np.array([float(col.mean()), float(col.std(ddof=0))], dtype=float))

    # global mean & std
    g_mean = atom.mean(axis=0) if atom.size else np.zeros(D_node, dtype=float)
    g_std  = atom.std(axis=0, ddof=0) if atom.size else np.zeros(D_node, dtype=float)
    parts.append(g_mean); parts.append(g_std)

    vec = np.concatenate(parts).astype(np.float32)
    if vec.size != D_total:
        raise ValueError(f"vec size mismatch {vec.size} vs D_total {D_total} at sample {i}")
    X[i,:] = vec
    y[i] = float(target)

    if (i+1) % 500 == 0 or (i+1) == N:
        print(f"  vectorized {i+1}/{N}")

t1 = time.time()
print("Vectorization time (s):", t1-t0)

# 5) Save X,y and mapping
np.save('cgvec_mapping/X_cgvecpp.npy', X)
np.save('cgvec_mapping/y_cgvecpp.npy', y)
with open('cgvec_mapping/per_sample_topk_records.json','w') as f:
    json.dump([], f)  # empty list since we did not compute top-K slots in this minimal run
print("Saved X,y and empty per_sample_topk_records (top-K not used).")


D_node= 37 num_rbf= 81 inferred D_edge= 81
D_total= 6386 mapping entries= 165
Saved mapping_global.json
  vectorized 500/3000
  vectorized 1000/3000
  vectorized 1500/3000
  vectorized 2000/3000
  vectorized 2500/3000
  vectorized 3000/3000
Vectorization time (s): 45.8219211101532
Saved X,y and empty per_sample_topk_records (top-K not used).


Feature Analysis

In [4]:
# ----------------------------
# Cell C: Train RF + SHAP + correlation analysis
# ----------------------------
import os, json, time
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr, spearmanr
import shap

# ---------- SETTINGS ----------
X_path = 'cgvec_mapping/X_cgvecpp.npy'
y_path = 'cgvec_mapping/y_cgvecpp.npy'
mapping_json = 'cgvec_mapping/mapping_global.json'
out_dir = 'cgvec_mapping'
rf_file = os.path.join(out_dir, 'rf_cgvecpp.joblib')
n_estimators = 200
random_state = 42

# SHAP settings
shap_bg_size = 200    # background size for TreeExplainer
shap_eval_size = 500  # number of test samples to compute SHAP for (subset)

# ---------- 0. Load data & mapping ----------
X = np.load(X_path)
y = np.load(y_path)
N, D_total = X.shape
print(f"Loaded X: {X.shape}, y: {y.shape}")

with open(mapping_json, 'r') as f:
    mapping_global = json.load(f)

# Build per-index mapping (feature_index -> meaning, block_type)
rows = []
for m in mapping_global:
    start = m['start']; end = m['end']
    for fi in range(start, end):
        rows.append({'feature_index': fi, 'block_type': m['type'], 'meta': {k:v for k,v in m.items() if k not in ['start','end']}})
df_map = pd.DataFrame(rows).sort_values('feature_index').reset_index(drop=True)

# We must decode per-index "meaning" to human-readable descriptors.
# We infer D_node (per-atom) from the first species_block length; D_node = (block_len - 1)/2
# Find first species_block to infer D_node
species_blocks = [m for m in mapping_global if m['type']=='species_block']
if len(species_blocks) == 0:
    raise RuntimeError("No species_block entries found in mapping_global.json")
first_block = species_blocks[0]
block_len = first_block['end'] - first_block['start']
D_node = int((block_len - 1) // 2)
# We also infer number of RBF bins by counting edge_rbf_bin entries (if present)
rbf_bins = len([m for m in mapping_global if m['type']=='edge_rbf_bin'])
print(f"Inferred D_node={D_node}, rbf_bins={rbf_bins}")

# helper to decode atom feature name for a given atom-feature offset (0..D_node-1)
def decode_atom_feature_name(offset):
    # Your CIFData node layout: [magmom] + OFM(32) + featurizer(4)
    # magmom at index 0
    if offset == 0:
        return 'magmom'
    # OFM components 1..32 => offsets 1..32 (if D_node>=33)
    ofm_start = 1
    ofm_len = 32
    if ofm_start <= offset < ofm_start + ofm_len:
        return f'OFM_component_{offset - ofm_start}'
    # featurizer after OFM
    feat_start = ofm_start + ofm_len
    fea_names = ['atomic_mass_norm','atomic_number_norm','atomic_radius_norm','electron_affinity_norm']
    if offset >= feat_start and offset < feat_start + len(fea_names):
        return fea_names[offset - feat_start]
    # fallback
    return f'node_{offset}'

# Now build readable 'meaning' per feature_index using mapping_global metadata
meaning_list = []
for idx, row in df_map.iterrows():
    fi = int(row['feature_index'])
    block = row['block_type']
    meta = row['meta']
    start = next(m['start'] for m in mapping_global if m['start'] <= fi < m['end'])
    block_entry = next(m for m in mapping_global if m['start'] == start)
    offset = fi - block_entry['start']
    meaning = None
    if block == 'species_block':
        species = block_entry['species']
        if offset == 0:
            meaning = f"{species}__count"
        elif 1 <= offset <= D_node:
            atom_pos = offset - 1
            meaning = f"{species}__mean__{decode_atom_feature_name(atom_pos)}"
        else:
            atom_pos = offset - 1 - D_node
            meaning = f"{species}__std__{decode_atom_feature_name(atom_pos)}"
    elif block == 'edge_rbf_bin':
        rbf_idx = int(block_entry.get('rbf_idx', offset//2 if rbf_bins>0 else 0))
        if offset % 2 == 0:
            meaning = f"rbf_bin{rbf_idx}__mean"
        else:
            meaning = f"rbf_bin{rbf_idx}__std"
    elif block == 'global_atom_mean':
        atom_pos = offset
        meaning = f"global_mean__{decode_atom_feature_name(atom_pos)}"
    elif block == 'global_atom_std':
        atom_pos = offset
        meaning = f"global_std__{decode_atom_feature_name(atom_pos)}"
    else:
        meaning = f"{block}__offset{offset}"
    meaning_list.append(meaning)

df_map['meaning'] = meaning_list
# save mapping table per-index
df_map.to_csv(os.path.join(out_dir, 'feature_index_meaning.csv'), index=False)
print("Saved feature_index_meaning.csv")

# ---------- 1. Stratified 60/20/20 split like original ----------
def make_safe_stratification_bins(y, min_per_bin=30, max_bins=10):
    for n_bins in range(max_bins, 1, -1):
        try:
            disc = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
            y_binned = disc.fit_transform(y.reshape(-1,1)).astype(int).ravel()
            _, counts = np.unique(y_binned, return_counts=True)
            if counts.min() >= min_per_bin:
                print(f"Using {n_bins} quantile bins for stratified split (min bin size {counts.min()})")
                return y_binned
        except Exception:
            continue
    raise ValueError("Could not bin targets for stratified split")

y_binned = make_safe_stratification_bins(y, min_per_bin=30, max_bins=10)
idx = np.arange(N)
train_idx, temp_idx, _, temp_bins = train_test_split(idx, y_binned, test_size=0.4, stratify=y_binned, random_state=random_state)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=temp_bins, random_state=random_state)

X_train, X_val, X_test = X[train_idx], X[val_idx], X[test_idx]
y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]
print("Split sizes -> train:", len(X_train), "val:", len(X_val), "test:", len(X_test))

# ---------- 2. Train RandomForest (original-style) ----------
rf = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state, n_jobs=-1)
t0 = time.time()
rf.fit(X_train, y_train)
t1 = time.time()
joblib.dump(rf, rf_file)
print(f"RF trained in {t1-t0:.2f}s and saved to {rf_file}")

# Evaluate
def evaluate(model, Xs, ys):
    preds = model.predict(Xs)
    return {
        'R2': r2_score(ys, preds),
        'MAE': mean_absolute_error(ys, preds),
        'RMSE': mean_squared_error(ys, preds, squared=False),
        'CC': pearsonr(ys, preds)[0]
    }

val_metrics = evaluate(rf, X_val, y_val)
test_metrics = evaluate(rf, X_test, y_test)
print("Validation metrics:", val_metrics)
print("Test metrics:", test_metrics)

# ---------- 3. Impurity-based importances (per-index) ----------
fi = rf.feature_importances_
df_imp = df_map[['feature_index','meaning','block_type']].copy()
df_imp['importance_impurity'] = fi[df_imp['feature_index'].values]
df_imp.to_csv(os.path.join(out_dir, 'feature_importances_vector_index.csv'), index=False)
print("Saved feature_importances_vector_index.csv")

# Aggregate impurity importance by physical meaning (sum across identical meanings)
agg_imp = df_imp.groupby('meaning', as_index=False)['importance_impurity'].sum().sort_values('importance_impurity', ascending=False)
agg_imp.to_csv(os.path.join(out_dir, 'importance_by_meaning_impurity.csv'), index=False)
print("Saved importance_by_meaning_impurity.csv")

# ---------- 4. SHAP (TreeExplainer) -> mean absolute SHAP per index -> aggregate by meaning ----------
# Create background sample from TRAIN for explainer
# ===== REPLACEMENT SHAP BLOCK (robust to additivity errors) =====
import warnings

# SHAP settings already defined earlier: shap_bg_size, shap_eval_size
bg_idx = np.random.RandomState(random_state).choice(len(X_train), size=min(shap_bg_size, len(X_train)), replace=False)
bg = X_train[bg_idx]

# Preferred: interventional TreeExplainer (more robust for correlated features)
# If this still raises an additivity error we fall back to check_additivity=False.
try:
    explainer = shap.TreeExplainer(rf, data=bg, feature_perturbation="interventional")
    # compute shap values (TreeExplainer returns array-like)
    X_eval = X_test[np.random.RandomState(random_state+1).choice(len(X_test), size=min(shap_eval_size, len(X_test)), replace=False)]
    print("Computing SHAP values with TreeExplainer(feature_perturbation='interventional') on eval subset (size {})...".format(X_eval.shape[0]))
    shap_vals = explainer.shap_values(X_eval)   # may be numpy array or list
except Exception as e:
    warnings.warn(f"Interventional explainer failed with: {e}. Falling back to check_additivity=False with default perturbation.")
    explainer = shap.TreeExplainer(rf, data=bg)  # default (tree path) explainer
    X_eval = X_test[np.random.RandomState(random_state+1).choice(len(X_test), size=min(shap_eval_size, len(X_test)), replace=False)]
    try:
        shap_vals = explainer.shap_values(X_eval, check_additivity=False)
    except Exception as e2:
        # Last resort: compute SHAP per-sample in a loop with check_additivity=False to avoid large memory peaks
        warnings.warn(f"Batch SHAP failed with: {e2}. Falling back to per-sample SHAP computation (slower).")
        shap_list = []
        for xi in X_eval:
            sv = explainer.shap_values(xi.reshape(1,-1), check_additivity=False)
            shap_list.append(np.array(sv).reshape(-1))
        shap_vals = np.vstack(shap_list)

# shap_vals should now be a (n_eval, n_features) numpy array for regression
shap_arr = np.array(shap_vals)  # ensure np.array type
if shap_arr.ndim == 3:
    # shap_values can sometimes be (n_eval, n_features, 1) for regression with some versions
    shap_arr = shap_arr.reshape(shap_arr.shape[0], shap_arr.shape[1])

# Aggregate mean absolute SHAP per feature index
mean_abs_shap = np.mean(np.abs(shap_arr), axis=0)

# Save per-index SHAP to CSV (same format as before)
df_shap = df_map[['feature_index','meaning','block_type']].copy()
df_shap['mean_abs_shap'] = mean_abs_shap[df_shap['feature_index'].values]
df_shap.to_csv(os.path.join(out_dir, 'feature_importances_with_shap.csv'), index=False)
print("Saved feature_importances_with_shap.csv (robust SHAP)")

# Aggregate SHAP by meaning and save
agg_shap = df_shap.groupby('meaning', as_index=False)['mean_abs_shap'].sum().sort_values('mean_abs_shap', ascending=False)
agg_shap.to_csv(os.path.join(out_dir, 'importance_by_meaning_shap.csv'), index=False)
print("Saved importance_by_meaning_shap.csv (robust SHAP)")

print("\nTop-20 SHAP-aggregated meanings (robust):")
print(agg_shap.head(20).to_string(index=False))
# ===== end replacement SHAP block =====

# ---------- 5. Correlation analysis (per-index) ----------
pearson_r = []
pearson_p = []
spearman_r = []
spearman_p = []
for fi_idx in df_map['feature_index'].values:
    col = X[:,fi_idx]
    try:
        r, p = pearsonr(col, y)
    except Exception:
        r, p = np.nan, np.nan
    try:
        rho, pr = spearmanr(col, y)
    except Exception:
        rho, pr = np.nan, np.nan
    pearson_r.append(r); pearson_p.append(p)
    spearman_r.append(rho); spearman_p.append(pr)

df_corr = df_map[['feature_index','meaning','block_type']].copy()
df_corr['pearson_r'] = pearson_r
df_corr['pearson_p'] = pearson_p
df_corr['spearman_rho'] = spearman_r
df_corr['spearman_p'] = spearman_p
df_corr.to_csv(os.path.join(out_dir, 'feature_correlations_index.csv'), index=False)
print("Saved feature_correlations_index.csv")

# Also aggregate correlations to a per-meaning measure by taking max absolute Pearson among indices sharing meaning (conservative)
agg_corr = df_corr.copy()
agg_corr['abs_pearson'] = np.abs(agg_corr['pearson_r'])
agg_corr_summary = agg_corr.groupby('meaning', as_index=False)['abs_pearson'].max().sort_values('abs_pearson', ascending=False)
agg_corr_summary.to_csv(os.path.join(out_dir, 'importance_by_meaning_correlation.csv'), index=False)
print("Saved importance_by_meaning_correlation.csv")

# ---------- 6. Print top-10 correlations (meaning-level) ----------
print("\nTop-10 meanings by max |Pearson r|:")
print(agg_corr_summary.head(10).to_string(index=False))

# Done
print("\nAll results saved to:", out_dir)


c:\Users\Sand\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded X: (3000, 6386), y: (3000,)
Inferred D_node=37, rbf_bins=81
Saved feature_index_meaning.csv
Using 10 quantile bins for stratified split (min bin size 300)
Split sizes -> train: 1800 val: 600 test: 600
RF trained in 6.32s and saved to cgvec_mapping\rf_cgvecpp.joblib
Validation metrics: {'R2': 0.8290945465844328, 'MAE': 0.010229092457254106, 'RMSE': 0.015034421346184676, 'CC': 0.9110879481778826}
Test metrics: {'R2': 0.786030639499318, 'MAE': 0.01115738538253742, 'RMSE': 0.017026803451910343, 'CC': 0.8867419508969241}
Saved feature_importances_vector_index.csv
Saved importance_by_meaning_impurity.csv


c:\Users\Sand\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\Sand\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Computing SHAP values with TreeExplainer(feature_perturbation='interventional') on eval subset (size 500)...


 99%|===================| 496/500 [00:29<00:00]        C:\Users\Sand\AppData\Local\Temp\ipykernel_3756\3228824978.py:194: UserWarning: Interventional explainer failed with: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. This check failed because for one of the samples the sum of the SHAP values was 0.047919, while the model output was 0.049662. If this difference is acceptable you can set check_additivity=False to disable this check.. Falling back to check_additivity=False with default perturbation.
  warnings.warn(f"Interventional explainer failed with: {e}. Falling back to check_additivity=False with default perturbation.")
 97%|=================== | 486/500 [00:29<00:00]       

Saved feature_importances_with_shap.csv (robust SHAP)
Saved importance_by_meaning_shap.csv (robust SHAP)

Top-20 SHAP-aggregated meanings (robust):
                            meaning  mean_abs_shap
      global_mean__OFM_component_13       0.017346
                global_mean__magmom       0.006488
      global_mean__OFM_component_14       0.002612
                    rbf_bin25__mean       0.002054
       global_std__OFM_component_13       0.001367
       global_std__OFM_component_12       0.001043
                    rbf_bin20__mean       0.000962
      global_mean__OFM_component_12       0.000958
       global_std__OFM_component_24       0.000817
global_mean__electron_affinity_norm       0.000730
                    rbf_bin19__mean       0.000725
                          Fe__count       0.000643
                     rbf_bin19__std       0.000585
 global_std__electron_affinity_norm       0.000538
      global_mean__OFM_component_24       0.000496
                    rbf_bin21__mean 

C:\Users\Sand\AppData\Local\Temp\ipykernel_3756\3228824978.py:240: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = pearsonr(col, y)
C:\Users\Sand\AppData\Local\Temp\ipykernel_3756\3228824978.py:244: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pr = spearmanr(col, y)


Saved feature_correlations_index.csv
Saved importance_by_meaning_correlation.csv

Top-10 meanings by max |Pearson r|:
                            meaning  abs_pearson
      global_mean__OFM_component_13     0.635680
        Fe__std__atomic_number_norm     0.589942
                          Fe__count     0.564021
                global_mean__magmom     0.534105
          Fe__std__atomic_mass_norm     0.526959
    Fe__std__electron_affinity_norm     0.517485
global_mean__electron_affinity_norm     0.432742
                   Fe__mean__magmom     0.379303
         Fe__mean__atomic_mass_norm     0.379303
       Fe__mean__atomic_number_norm     0.379303

All results saved to: cgvec_mapping


Feature Analysis plots:

In [8]:
# ----------------------------
# Plotting cell (run after Cell C success)
# Generates paper-ready figures and saves them to cgvec_mapping/figures/
# ----------------------------
import os, json, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from scipy.stats import pearsonr
import joblib

# --------- Files and folders ----------
out_dir = 'cgvec_mapping'
fig_dir = os.path.join(out_dir, 'figures')
os.makedirs(fig_dir, exist_ok=True)

# Required files (created by previous cells)
files = {
    'X': os.path.join(out_dir, 'X_cgvecpp.npy'),
    'y': os.path.join(out_dir, 'y_cgvecpp.npy'),
    'map': os.path.join(out_dir, 'feature_index_meaning.csv'),
    'impurity': os.path.join(out_dir, 'feature_importances_vector_index.csv'),
    'shap_index': os.path.join(out_dir, 'feature_importances_with_shap.csv'),
    'agg_shap': os.path.join(out_dir, 'importance_by_meaning_shap.csv'),
    'agg_impurity': os.path.join(out_dir, 'importance_by_meaning_impurity.csv'),
    'corr_index': os.path.join(out_dir, 'feature_correlations_index.csv'),
    'rf_model': os.path.join(out_dir, 'rf_cgvecpp.joblib')
}

# Quick existence check
for k,v in files.items():
    if not os.path.exists(v):
        print(f"WARNING: missing file {k}: {v}  (some plots may be skipped)")
print("Files checked. Proceeding to load available data...")

# Load arrays
X = np.load(files['X'])
y = np.load(files['y'])
N, D = X.shape
print("Loaded X,y shapes:", X.shape, y.shape)

# Load mapping, importance, shap, correlations (if present)
df_map = pd.read_csv(files['map']) if os.path.exists(files['map']) else None
df_imp_idx = pd.read_csv(files['impurity']) if os.path.exists(files['impurity']) else None
df_shap_idx = pd.read_csv(files['shap_index']) if os.path.exists(files['shap_index']) else None
df_agg_shap = pd.read_csv(files['agg_shap']) if os.path.exists(files['agg_shap']) else None
df_agg_imp = pd.read_csv(files['agg_impurity']) if os.path.exists(files['agg_impurity']) else None
df_corr = pd.read_csv(files['corr_index']) if os.path.exists(files['corr_index']) else None
rf = joblib.load(files['rf_model']) if os.path.exists(files['rf_model']) else None

# Load test split predictions: if you trained earlier and still have splits in memory it's better.
# If not available, we will recompute predictions by splitting dataset again reproducibly.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
def make_safe_stratification_bins(y, min_per_bin=30, max_bins=10):
    for n_bins in range(max_bins, 1, -1):
        try:
            disc = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
            y_binned = disc.fit_transform(y.reshape(-1,1)).astype(int).ravel()
            _, counts = np.unique(y_binned, return_counts=True)
            if counts.min() >= min_per_bin:
                return y_binned
        except Exception:
            continue
    raise ValueError("Could not bin targets for stratified split")

y_binned = make_safe_stratification_bins(y, min_per_bin=30, max_bins=10)
idx = np.arange(N)
train_idx, temp_idx, _, temp_bins = train_test_split(idx, y_binned, test_size=0.4, stratify=y_binned, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=temp_bins, random_state=42)

X_test = X[test_idx]; y_test = y[test_idx]
print("Using reproducible splits. Test size:", len(X_test))

# Predictions
if rf is not None:
    y_pred_test = rf.predict(X_test)
else:
    raise RuntimeError("RF model not found; cannot produce True vs Predicted plot.")

# ---------- Fig 1: True vs Predicted (Test) ----------
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test, alpha=0.6, edgecolors='k', linewidth=0.2)
mn = min(y_test.min(), y_pred_test.min()); mx = max(y_test.max(), y_pred_test.max())
plt.plot([mn, mx], [mn, mx], 'r--', lw=1.5)
plt.xlabel('True Magnetization')
plt.ylabel('Predicted Magnetization')
plt.title('True vs Predicted Magnetization (Test)')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, 'true_vs_pred_test.png'), dpi=300)
plt.savefig(os.path.join(fig_dir, 'true_vs_pred_test.pdf'))
plt.close()
# numeric metrics for caption
r2 = rf.score(X_test, y_test)
cc = pearsonr(y_test, y_pred_test)[0]
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, y_pred_test)
rmse = mean_squared_error(y_test, y_pred_test, squared=False)
print("Fig1 saved. Test metrics: R2={:.4f}, CC={:.4f}, MAE={:.5f}, RMSE={:.5f}".format(r2, cc, mae, rmse))

# ---------- Helper: select and prepare top-N by SHAP or impurity ----------
def top_meaning_df(df_agg, topn=20):
    df = df_agg.copy()
    df = df.sort_values(df.columns[1], ascending=False).head(topn)  # second column assumed to be importance
    # rename cols for convenience
    df.columns = ['meaning','importance']
    return df

# ---------- Fig 2: Top-20 meanings by SHAP (aggregated) ----------
if df_agg_shap is not None:
    df_top_shap = top_meaning_df(df_agg_shap, topn=20)
    plt.figure(figsize=(8,6))
    plt.barh(np.arange(len(df_top_shap)), df_top_shap['importance'][::-1])
    plt.yticks(np.arange(len(df_top_shap)), df_top_shap['meaning'][::-1], fontsize=8)
    plt.xlabel('Aggregated mean(|SHAP|)')
    plt.title('Top-20 descriptors by SHAP importance (aggregated)')
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'top20_meaning_shap.png'), dpi=300)
    plt.savefig(os.path.join(fig_dir, 'top20_meaning_shap.pdf'))
    plt.close()
    print("Fig2 saved: top20_meaning_shap")
else:
    print("SHAP aggregated CSV not found; skipping Fig2 (top SHAP).")

# ---------- Fig 3: Top-20 meanings by impurity importance ----------
if df_agg_imp is not None:
    df_top_imp = top_meaning_df(df_agg_imp, topn=20)
    plt.figure(figsize=(8,6))
    plt.barh(np.arange(len(df_top_imp)), df_top_imp['importance'][::-1])
    plt.yticks(np.arange(len(df_top_imp)), df_top_imp['meaning'][::-1], fontsize=8)
    plt.xlabel('Aggregated impurity importance')
    plt.title('Top-20 descriptors by impurity importance (aggregated)')
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'top20_meaning_impurity.png'), dpi=300)
    plt.savefig(os.path.join(fig_dir, 'top20_meaning_impurity.pdf'))
    plt.close()
    print("Fig3 saved: top20_meaning_impurity")
else:
    print("Impurity aggregated CSV not found; skipping Fig3.")

# ---------- Fig 4: Correlation heatmap for selected descriptors (OFM, magmom, rbf bins) ----------
if df_corr is not None:
    # choose descriptors to include: OFM components and magmom and top rbf bins by abs Pearson
    # find all meanings that include OFM_component or magmom or rbf_bin
    sel_mask = df_corr['meaning'].str.contains('OFM_component|__magmom|rbf_bin')
    df_sel = df_corr[sel_mask].copy()
    # pick top K by abs Pearson
    df_sel['abs_pearson'] = df_sel['pearson_r'].abs()
    df_sel_top = df_sel.sort_values('abs_pearson', ascending=False).head(30)
    # build matrix of values across samples for these meanings (extract column indices of the meaning)
    sel_indices = df_sel_top['feature_index'].values.astype(int).tolist()
    data_mat = X[:, sel_indices]
    # compute correlation matrix among selected descriptors (not to be confused with corr with target)
    corrmat = np.corrcoef(data_mat.T)
    names = df_sel_top['meaning'].values
    plt.figure(figsize=(10,8))
    sns.heatmap(corrmat, xticklabels=names, yticklabels=names, vmax=1.0, vmin=-1.0, cmap='coolwarm', center=0)
    plt.title('Descriptor correlation matrix (top 30 OFM/magmom/rbf descriptors)')
    plt.xticks(rotation=90, fontsize=7)
    plt.yticks(rotation=0, fontsize=7)
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'descriptor_corr_heatmap.png'), dpi=300)
    plt.savefig(os.path.join(fig_dir, 'descriptor_corr_heatmap.pdf'))
    plt.close()
    print("Fig4 saved: descriptor_corr_heatmap")
else:
    print("Correlation CSV not found; skipping Fig4.")

# ---------- Fig 5: SHAP beeswarm for top per-index features (if shap per-index array exists) ----------
# Try to load raw shap values used earlier from 'explainer' step if saved; else skip.
shap_idx_path = os.path.join(out_dir, 'shap_values_eval.npy')
if os.path.exists(shap_idx_path) and df_shap_idx is not None:
    shap_vals_full = np.load(shap_idx_path)  # shape (n_eval, n_features)
    # get top per-index by mean_abs_shap
    top_idx = df_shap_idx.sort_values('mean_abs_shap', ascending=False).head(30)['feature_index'].values.astype(int)
    import shap as _shap
    plt.figure(figsize=(8,6))
    _shap.summary_plot(shap_vals_full[:, top_idx], X_test[:, top_idx], plot_type='dot', show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, 'shap_beeswarm_top30.png'), dpi=300)
    plt.close()
    print("Fig5 saved: shap_beeswarm_top30")
else:
    # fallback: if df_shap_idx present but raw shap array not saved, produce bar chart of per-index SHAP top30 instead
    if df_shap_idx is not None:
        df_top30_idx = df_shap_idx.sort_values('mean_abs_shap', ascending=False).head(30)
        plt.figure(figsize=(8,6))
        plt.barh(range(len(df_top30_idx)), df_top30_idx['mean_abs_shap'][::-1])
        plt.yticks(range(len(df_top30_idx)), df_top30_idx['meaning'][::-1], fontsize=7)
        plt.xlabel('mean(|SHAP|)')
        plt.title('Top-30 per-index features by mean(|SHAP|)')
        plt.tight_layout()
        plt.savefig(os.path.join(fig_dir, 'shap_top30_bar.png'), dpi=300)
        plt.close()
        print("Fig5 saved (bar fallback): shap_top30_bar")
    else:
        print("SHAP per-index CSV not found; skipping Fig5.")

# ---------- Fig 6: Per-species aggregated SHAP importance (if multiple species present) ----------
# ---- Replacement Fig6: show TOP-10 species only (clean, readable) ----
if df_agg_shap is not None:
    df = df_agg_shap.copy()

    # same species-extraction heuristic as before
    def species_from_meaning(s):
        p = s.split('__')[0]
        if p in ('global_mean', 'global_std') or p.startswith('rbf_bin'):
            return 'GLOBAL'
        if len(p) <= 3 and p.isalpha():
            return p
        return 'OTHER'

    df['species'] = df['meaning'].apply(species_from_meaning)
    species_group = df.groupby('species')['mean_abs_shap'].sum().reset_index()
    species_group = species_group.sort_values('mean_abs_shap', ascending=False)

    topN = 10
    df_top = species_group.head(topN)

    plt.figure(figsize=(10,4))
    import seaborn as sns
    sns.barplot(x='species', y='mean_abs_shap', data=df_top, palette='muted')
    plt.xlabel('Species or GLOBAL', fontsize=11)
    plt.ylabel('Sum mean(|SHAP|)', fontsize=11)
    plt.title('Aggregated SHAP by species (top 10)', fontsize=13)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.tight_layout()
    out_path = os.path.join(fig_dir, 'shap_by_species_top10.png')
    plt.savefig(out_path, dpi=300)
    plt.close()
    print("Fig6 saved:", out_path)
else:
    print("Aggregated SHAP CSV missing; skipping Fig6.")



Files checked. Proceeding to load available data...
Loaded X,y shapes: (3000, 6386) (3000,)
Using reproducible splits. Test size: 600


c:\Users\Sand\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Fig1 saved. Test metrics: R2=0.7860, CC=0.8867, MAE=0.01116, RMSE=0.01703
Fig2 saved: top20_meaning_shap
Fig3 saved: top20_meaning_impurity
Fig4 saved: descriptor_corr_heatmap
Fig5 saved (bar fallback): shap_top30_bar


C:\Users\Sand\AppData\Local\Temp\ipykernel_3756\3786483809.py:225: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='species', y='mean_abs_shap', data=df_top, palette='muted')


Fig6 saved: cgvec_mapping\figures\shap_by_species_top10.png


In [6]:
# ----------------------------
# Code 5 (REVISED):
# Clean, paper-ready plots for Top-20 CGVEC descriptors
# - Expanded x-axis (2× max importance)
# - Structured y-axis labels: FeatureGroup[dim] → Findex / Ftotal
# ----------------------------

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- Paths ----------
out_dir = 'cgvec_mapping'
fig_dir = os.path.join(out_dir, 'figures')
os.makedirs(fig_dir, exist_ok=True)

# ---------- Load data ----------
df_shap = pd.read_csv(os.path.join(out_dir, 'importance_by_meaning_shap.csv'))
df_imp  = pd.read_csv(os.path.join(out_dir, 'importance_by_meaning_impurity.csv'))
df_map  = pd.read_csv(os.path.join(out_dir, 'feature_index_meaning.csv'))

D_total = df_map['feature_index'].max() + 1  # total vector length

# ---------- Helper: clean feature label ----------
def format_label(meaning):
    """
    Converts:
    global_mean OFM[13] (idx 6326)
    → OFM[13] (F6326 / 6386)
    """
    row = df_map[df_map['meaning'] == meaning]
    if len(row) == 0:
        return meaning

    idx = int(row.iloc[0]['feature_index'])

    if 'OFM_component_' in meaning:
        k = meaning.split('OFM_component_')[-1]
        return f'OFM[{k}]  (F{idx}/{D_total})'

    if '__magmom' in meaning:
        return f'magmom  (F{idx}/{D_total})'

    if 'rbf_bin' in meaning:
        bin_id = meaning.split('rbf_bin')[-1].split('__')[0]
        stat = 'mean' if 'mean' in meaning else 'std'
        return f'RBF[{bin_id}] {stat}  (F{idx}/{D_total})'

    if 'atomic_' in meaning:
        name = meaning.split('__')[-1].replace('_norm', '')
        return f'{name}  (F{idx}/{D_total})'

    return f'{meaning}  (F{idx}/{D_total})'


# ---------- Generic plotting function ----------
def plot_top20(df, value_col, title, xlabel, filename):
    df_top = df.sort_values(value_col, ascending=False).head(20).copy()
    df_top['label'] = df_top['meaning'].apply(format_label)

    values = df_top[value_col].values
    labels = df_top['label'].values

    xmax = values.max() * 2.0  # 🔑 double x-axis range

    plt.figure(figsize=(9, 6))
    plt.barh(range(len(values)), values[::-1])
    plt.yticks(range(len(labels)), labels[::-1], fontsize=9)
    plt.xlabel(xlabel)
    plt.title(title)
    plt.xlim(0, xmax)
    plt.tight_layout()

    plt.savefig(os.path.join(fig_dir, filename + '.png'), dpi=300)
    plt.savefig(os.path.join(fig_dir, filename + '.pdf'))
    plt.close()


# ---------- Plot 1: SHAP ----------
plot_top20(
    df_shap,
    value_col='mean_abs_shap',
    title='Top-20 CGVEC descriptors by SHAP importance',
    xlabel='Aggregated mean(|SHAP|)',
    filename='top20_cgvec_shap_clean'
)

# ---------- Plot 2: Impurity ----------
plot_top20(
    df_imp,
    value_col='importance_impurity',
    title='Top-20 CGVEC descriptors by impurity importance',
    xlabel='Aggregated impurity importance',
    filename='top20_cgvec_impurity_clean'
)

print("✅ Clean Top-20 SHAP and impurity plots saved to:", fig_dir)


✅ Clean Top-20 SHAP and impurity plots saved to: cgvec_mapping\figures


In [9]:
# =========================================================
# Top-20 CGVEC SHAP plot (aggregated)
# =========================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------- Paths --------
base_dir = "cgvec_mapping"
shap_file = os.path.join(base_dir, "importance_by_meaning_shap.csv")
map_file  = os.path.join(base_dir, "feature_index_meaning.csv")
out_fig   = os.path.join(base_dir, "top20_cgvec_shap_clean.pdf")

# -------- Load data --------
df_shap = pd.read_csv(shap_file)
df_map  = pd.read_csv(map_file)

# -------- Take top-20 unique meanings --------
df_top = (
    df_shap
    .sort_values("mean_abs_shap", ascending=False)
    .head(20)
    .reset_index(drop=True)
)

# -------- Map each meaning to a representative F-index --------
# (first occurrence in the vector)
meaning_to_f = (
    df_map
    .groupby("meaning")["feature_index"]
    .min()
    .to_dict()
)

df_top["F_label"] = df_top["meaning"].map(
    lambda m: f"F{meaning_to_f[m]:03d}"
)

# -------- Short, SAFE physics-aware names --------
def short_name(m):
    if "OFM_component_12" in m:
        return "d⁵ valence occupancy"
    if "OFM_component_13" in m:
        return "d⁶ valence occupancy"
    if "OFM_component_14" in m:
        return "d⁷ valence occupancy"
    if "OFM_component_24" in m:
        return "f⁷ valence occupancy"
    if "magmom" in m and "mean" in m:
        return "mean atomic spin moment"
    if "magmom" in m and "std" in m:
        return "variation of atomic spin moments"
    if "rbf_bin" in m and "__mean" in m:
        k = int(m.split("rbf_bin")[1].split("__")[0])
        return f"mean neighbor contribution (~{k*0.1:.1f} Å)"
    if "rbf_bin" in m and "__std" in m:
        k = int(m.split("rbf_bin")[1].split("__")[0])
        return f"variation of neighbor contribution (~{k*0.1:.1f} Å)"
    if "__count" in m:
        return "species atom count"
    if "atomic_number_norm" in m and "mean" in m:
        return "mean atomic number"
    if "atomic_number_norm" in m and "std" in m:
        return "variation in atomic number"

    if "atomic_radius_norm" in m and "mean" in m:
        return "mean atomic radius"
    if "atomic_radius_norm" in m and "std" in m:
         return "variation in atomic radius"

    if "electron_affinity_norm" in m and "mean" in m:
        return "mean electron affinity"
    if "electron_affinity_norm" in m and "std" in m:
        return "variation in electron affinity"


df_top["short_name"] = df_top["meaning"].apply(short_name)

# -------- Plot --------
plt.figure(figsize=(8, 6))

y_pos = np.arange(len(df_top))

plt.barh(
    y_pos,
    df_top["mean_abs_shap"],
    color="#4C72B0"
)

# Y-axis labels: F-indices only
plt.yticks(y_pos, df_top["F_label"], fontsize=10)
plt.gca().invert_yaxis()

# X-axis
plt.xlabel("Average feature contribution (SHAP)", fontsize=12)
plt.xlim(0.0, 0.023)

# Add short feature names ABOVE bars
for i, (val, name) in enumerate(zip(df_top["mean_abs_shap"], df_top["short_name"])):
    plt.text(
        val + 0.0004,
        i,
        name,
        va="center",
        fontsize=9
    )

plt.title("Top 20 CGVEC Features by SHAP Importance", fontsize=13)
plt.tight_layout()

plt.savefig(out_fig)
plt.savefig(out_fig.replace(".pdf", ".png"), dpi=300)
plt.close()

print("Saved:", out_fig)


Saved: cgvec_mapping\top20_cgvec_shap_clean.pdf
